In [3]:
# Appending the directory containing your `python_startup` file, with credentials filled out
import sys
sys.path.append('/Users/Myers/Documents/GitHub/myers2022/Python/Jupyter/python_startup.py')
from python_startup import * 

from tqdm import tqdm
from collections import Counter

# Calculation of multi-step effective connectivity, from adult MB FIB paper

* In several analyses, we computed the “effective” connectivity through multi-synaptic pathways between a set of source and target neurons: Figure 10-figure supplement 2; Figure 11-figure supplement 1; Figure 26-figure supplement 1 and 2; Figure 27-figure supplement 3. 

* Although our procedure generalizes to pathways of any length, we only performed it for two-step (or “one-hop”) pathways. 

* To do so, we determined the set of interneurons either postsynaptic to the source population or presynaptic to the target population. 

* Starting with the matrices of source-interneuron connectivity and interneuron-target connectivity, we normalized each so that the sum of inputs to each postsynaptic cell summed to  1. 

* Then we multiplied the two matrices to yield an estimate of effective source-target connectivity. 

* This procedure reflects the assumption that an output synapse from an interneuron conveys information about its inputs to varying degrees, which are proportional to the number of input synapses coming from each input.

#Philip: I want to expand this to an analysis of all uPNs to all DNs. Ideally there will be multiple outputs for different parameters within the analysis. Should we confine our interneuron population to LHNs? Or just all downstream targets of uPNs? How do we maintain glomerular identity in the final analysis? How do we treat cell types? Worth having a look at the MB FIB paper figures referenced above for this. 

# Getting uPNs

This is a bunch of python code to call subsets of neurons. We should start with our defined uPNs, either filtering out mPNs for the time being, or treating them as a block (in the first analysis). While I do this I'll be keeping track of useful bits of code for exploring data in python, which I've largely forgotten. So first, how to look at the dataset we've loaded and figure out what's going on. Use https://connectome-neuprint.github.io/neuprint-python/docs/notebooks/QueryTutorial.html for guidance. And: https://connectome-neuprint.github.io/neuprint-python/docs/index.html. You can use the neuprint website to get an idea of how the data is organised. 

Common queries is a very useful resource: https://connectome-neuprint.github.io/neuprint-python/docs/queries.html#queries

### Fetching the PNs - at this stage, just everything that contains 'PN' - this can be refined later

In [4]:
from neuprint import NeuronCriteria as NC

q = """MATCH (m:`hemibrain_Meta`) WITH m.superLevelRois AS rois MATCH (neuron :`hemibrain_Neuron` {`AL(R)`: true}) WHERE (neuron.type CONTAINS "PN" OR neuron.instance CONTAINS "PN")    
        RETURN neuron.bodyId AS bodyid, neuron.instance AS bodyname, neuron.type AS bodytype, neuron.status AS neuronStatus, neuron.roiInfo AS roiInfo, neuron.size AS size, neuron.pre AS npre, neuron.post AS npost, rois ORDER BY neuron.bodyId
"""
PN_df = neu.fetch_custom(q)

print(f"Found {len(PN_df)} results")

PN_df

Found 337 results


,bodyid,bodyname,bodytype,neuronStatus,roiInfo,size,npre,npost,rois
0,294792184,M_vPNml53_R,M_vPNml53,Traced,"{""SNP(R)"": {""pre"": 70, ""post"": 155, ""downstrea...",420662445,92,344,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."
1,329599710,M_lvPNm32_R,M_lvPNm32,Traced,"{""SNP(R)"": {""pre"": 180, ""post"": 93, ""downstrea...",343478957,247,285,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."
2,417199910,M_lvPNm36_R,M_lvPNm36,Traced,"{""SNP(R)"": {""pre"": 156, ""post"": 95, ""downstrea...",387058559,162,347,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."
3,480927537,M_vPNml70_R,M_vPNml70,Traced,"{""SNP(R)"": {""pre"": 15, ""post"": 18, ""downstream...",240153322,82,276,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."
4,481268653,M_vPNml89_R,M_vPNml89,Traced,"{""SNP(R)"": {""pre"": 10, ""post"": 2, ""downstream""...",265085609,146,58,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."
...,...,...,...,...,...,...,...,...,...
332,5813090752,M_vPNml81_R,M_vPNml81,Traced,"{""LH(R)"": {""pre"": 91, ""post"": 70, ""downstream""...",237647113,92,181,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."
333,5901194250,M_lvPNm37_R,M_lvPNm37,Traced,"{""SNP(R)"": {""pre"": 125, ""post"": 53, ""downstrea...",364231991,185,251,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."
334,5901194556,M_lvPNm37_R,M_lvPNm37,Traced,"{""SNP(R)"": {""pre"": 137, ""post"": 53, ""downstrea...",405048506,176,256,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."
335,5901222731,VP1d+VP4_l2PN2(lALT)_R,VP1d+VP4_l2PN2,Traced,"{""LH(R)"": {""pre"": 729, ""post"": 624, ""downstrea...",2904336647,2113,4087,"[ME(R), AME(R), LO(R), LOP(R), CA(R), CA(L), P..."


## From here there are two directions we can take - the simpler way using Philipp's functions, or the more involved (and therefore more customisable?) method used with Markus. First, let's see if we can make two matrices, of normalised input from PNs to LHLNs, and then from LHLNs, to DNs, using Philipp's relatively simple approach. The crucial distinction to keep in mind here is between cells and cell types

### Let's find all the downstream partners of the PNs, then filter by the output type (contains LH, but doesn't include LHCENTs, or something)

In [5]:
PN_bids = PN_df.bodyid.tolist()
len(PN_bids)

337

In [4]:
### Make a dictionary for bodyid to type

In [51]:
PN_to_type = dict(zip(PN_df.bodyid.tolist(), PN_df.bodytype.tolist()))

In [7]:
# Example: Fetch all downstream connections FROM a set of neurons

from neuprint import fetch_adjacencies, NeuronCriteria as NC

pn_out_neuron_df, pn_out_conn_df = fetch_adjacencies(PN_bids, None)



In [7]:
pn_out_conn_df

,bodyId_pre,bodyId_post,roi,weight
0,294792184,264438143,SLP(R),8
1,294792184,264822904,SLP(R),1
2,294792184,265120424,SLP(R),4
3,294792184,265120424,SIP(R),1
4,294792184,265120467,SLP(R),4
...,...,...,...,...
111250,5901222910,5901207528,CA(R),34
111251,5901222910,5901218894,AL(R),26
111252,5901222910,5901219179,LH(R),2
111253,5901222910,5901225361,CA(R),2


In [8]:
pn_to_lh_conn_df = pn_out_conn_df[pn_out_conn_df["roi"] == 'LH(R)']
pn_to_lh_conn_df

,bodyId_pre,bodyId_post,roi,weight
11,294792184,294786630,LH(R),1
17,294792184,295120601,LH(R),2
19,294792184,295127956,LH(R),1
30,294792184,295797312,LH(R),1
34,294792184,295814411,LH(R),2
...,...,...,...,...
111245,5901222910,5901193783,LH(R),1
111246,5901222910,5901194027,LH(R),12
111247,5901222910,5901194039,LH(R),2
111248,5901222910,5901203780,LH(R),1


 ### Or we could use Philipp's function to filter by output ROI, in this case, LH(R). Interested to see if this returns the same number of connection pairs...it does

In [8]:
#Fetch all direct connections between a set of upstream neurons and downstream neurons. 
#In this case, our list of bids that contain PN, and partners in the LH

pn_lh_neuron_df, pn_lh_conn_df = fetch_adjacencies(PN_bids, NC(rois=['LH(R)']))

In [10]:
pn_lh_conn_df

,bodyId_pre,bodyId_post,roi,weight
0,294792184,294436967,SLP(R),4
1,294792184,294760699,SLP(R),1
2,294792184,294786630,LH(R),1
3,294792184,295120601,LH(R),2
4,294792184,295127956,SLP(R),1
...,...,...,...,...
64296,5901222910,5901193783,LH(R),1
64297,5901222910,5901194027,LH(R),12
64298,5901222910,5901194039,LH(R),2
64299,5901222910,5901203780,LH(R),1


In [25]:
#filter by roi. 

pn_lh_conn_df = pn_lh_conn_df[pn_lh_conn_df["roi"] == 'LH(R)']
pn_lh_conn_df

,bodyId_pre,bodyId_post,roi,weight
2,294792184,294786630,LH(R),1
3,294792184,295120601,LH(R),2
5,294792184,295127956,LH(R),1
7,294792184,295797312,LH(R),1
9,294792184,295814411,LH(R),2
...,...,...,...,...
64296,5901222910,5901193783,LH(R),1
64297,5901222910,5901194027,LH(R),12
64298,5901222910,5901194039,LH(R),2
64299,5901222910,5901203780,LH(R),1


In [26]:
### From here we can use either dataframe, they are identical. 
pn_lh_conn_df.bodyId_post


2         294786630
3         295120601
5         295127956
7         295797312
9         295814411
            ...    
64296    5901193783
64297    5901194027
64298    5901194039
64299    5901203780
64300    5901219179
Name: bodyId_post, Length: 38812, dtype: int64

In [27]:
#We don't want any axo-axonic PN connectivity in the LH, so I'd like to remove any rows 
#where the bodyId_post matches any member of bodyId_pre. 

pn_lh_conn_df = pn_lh_conn_df[~pn_lh_conn_df.bodyId_post.isin(pn_lh_conn_df.bodyId_pre.tolist())]

In [28]:
pn_lh_conn_df

,bodyId_pre,bodyId_post,roi,weight
2,294792184,294786630,LH(R),1
3,294792184,295120601,LH(R),2
5,294792184,295127956,LH(R),1
7,294792184,295797312,LH(R),1
9,294792184,295814411,LH(R),2
...,...,...,...,...
64296,5901222910,5901193783,LH(R),1
64297,5901222910,5901194027,LH(R),12
64298,5901222910,5901194039,LH(R),2
64299,5901222910,5901203780,LH(R),1


In [29]:
#Merge arbitrary neuron properties onto the connection table with merge_neuron_properties()

from neuprint import merge_neuron_properties

pn_lh_conn_df = merge_neuron_properties(pn_out_neuron_df, pn_lh_conn_df, ['type', 'instance'])
pn_lh_conn_df

,bodyId_pre,bodyId_post,roi,weight,type_pre,instance_pre,type_post,instance_post
0,294792184,294786630,LH(R),1,M_vPNml53,M_vPNml53_R,LHPD4e1_b,LHPD4e1_b_R
1,294792184,295120601,LH(R),2,M_vPNml53,M_vPNml53_R,SLP352,SLP352_R
2,294792184,295127956,LH(R),1,M_vPNml53,M_vPNml53_R,SLP271,SLP271_R
3,294792184,295797312,LH(R),1,M_vPNml53,M_vPNml53_R,SLP346,SLP346_R
4,294792184,295814411,LH(R),2,M_vPNml53,M_vPNml53_R,SLP211,SLP211_R
...,...,...,...,...,...,...,...,...
36897,5901222910,5901193783,LH(R),1,DM2_lPN,DM2_lPN_R,LHAV4g4_a,LHAV4g4_a_R
36898,5901222910,5901194027,LH(R),12,DM2_lPN,DM2_lPN_R,LHAV3c3,LHAV3c3_R
36899,5901222910,5901194039,LH(R),2,DM2_lPN,DM2_lPN_R,LHAV4g2,LHAV4g2_R
36900,5901222910,5901203780,LH(R),1,DM2_lPN,DM2_lPN_R,LHAV4g11,LHAV4g11_R


In [166]:
#Just double-checking that the matrix below seems right

pn_to_lh_conn_df[pn_to_lh_conn_df['type_post'].str.contains('mAL4', na = False)]

,bodyId_pre,bodyId_post,roi,weight,type_pre,instance_pre,type_post,instance_post
25256,1851393805,609867847,LH(R),2,VM7d_adPN,VM7d_adPN_R,mAL4,mAL4A_L
25268,1851393805,666818214,LH(R),2,VM7d_adPN,VM7d_adPN_R,mAL4,mAL4A_L
32975,5813050727,666818214,LH(R),3,VM7d_adPN,VM7d_adPN_R,mAL4,mAL4A_L


In [11]:
#Not entirely sure what we've got by using the LH(R) roi for this filtering process. Maybe we should be 
#using something else instead?
#For example, how would we remove LHLNs? By the number of outputs they have within the LH? There is a lot more 
#filtering to do here because there is all kinds of crap. One obvious answer is to filter by number of 
#connections...just remove all the single synapses between types? 

#One approach is to filter by 'LH' in the type name, and then remove LHCENTs, but this is not the most
#sophisticated approach. How about, as Philipp mentions, useing fetch_neurons() to get some info on 
#whether these neurons have the majority of their postsynapses in the LH. Or we could just use Alex's 
#LHN list from R, imported in the same way we did with the DNs. 



# So really we want polarised neurons, in which case we should use Philipp's approach of fetching neurons, 
# then filtering the LHN list by those with a majority of their synapses in the LH. Let's try that immediately below

In [44]:
from neuprint import fetch_neurons

pn_lhns = pn_lh_conn_df.bodyId_post.unique()

pn_lhns_df, lh_roi_counts_df = fetch_neurons(pn_lhns)

In [45]:
len(pn_lhns)

3018

In [46]:
len(lh_roi_counts_df.bodyId)

19176

In [47]:
lh_roi_counts_df

,bodyId,roi,pre,post
0,263674097,SNP(R),121,330
1,263674097,SMP(R),93,252
2,263674097,SIP(R),28,72
3,263674097,LH(R),10,184
4,263674097,SLP(R),0,6
...,...,...,...,...
19171,7112615307,VLNP(R),3,2
19172,7112615307,PLP(R),3,2
19173,7112622144,LH(R),0,11
19174,7112623352,LH(R),2,12


### So in lh_roi_counts_df we have a list of bodyids and their rois...we want to filter this list for those cells which have the majority of their postsynapses (inputs) in the LH. How exactly might we do this? 

In [73]:
#The mega dumb way

#pn_to_lh_conn_df = pn_to_lh_conn_df[pn_to_lh_conn_df['type_post'].str.contains('LH', na = False)]

In [ ]:
#Or we could just filter by removing very weak connections. Remove columns with fewer than 10 PN inputs total?
#This is at least a bit less stupid than filtering by name contains LH. Let's try that below, on the matrix itself. 

In [92]:
from neuprint.utils import connection_table_to_matrix

pn_lh_matrix = connection_table_to_matrix(pn_to_lh_conn_df, 'type', sort_by='type')
pn_lh_matrix

type_post,5-HTPMPV03,AL-MBDL1,ALIN3,AOTU010,AOTU043,APL,AVLP003,AVLP004,AVLP010,AVLP012,...,mAL3A,mAL4,mAL5B,mAL6,mALB1,mALB2,mALB3,mALB4,mALD1,NaN
type_pre,,,,,,,,,,,,,,,,,,,,,
DA1_lPN,0,0,0,0,0,0,0,0,4,7,...,0,0,0,0,17,0,0,0,0,NaN
DA1_vPN,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
DA2_lPN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
DA3_adPN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
DA4l_adPN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VP5+Z_adPN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
VP5+_l2PN,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,NaN
V_ilPN,0,6,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN


In [118]:
#Like I said above, let's remove columns that sum to less than 10

pn_lh_matrix.T[np.array(pn_lh_matrix.sum(axis = 0) > 10)].T


type_post,AL-MBDL1,ALIN3,AVLP003,AVLP010,AVLP013,AVLP015,AVLP030,AVLP044_b,AVLP189_b,AVLP209,...,WEDPN6A,WEDPN7A,WEDPN7B,aMe24,aSP-f3,aSP-g2,mAL6,mALB1,mALB2,mALB3
type_pre,,,,,,,,,,,,,,,,,,,,,
DA1_lPN,0.0,0.0,0.0,4.0,4.0,0.0,6.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,37.0,0.0,17.0,0.0,0.0
DA1_vPN,0.0,0.0,0.0,0.0,0.0,10.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
DA2_lPN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
DA3_adPN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DA4l_adPN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VP5+Z_adPN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
VP5+_l2PN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
V_ilPN,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14.0,4.0,0.0,0.0,1.0,0.0,0.0


In [117]:
dir(nvneu)

['Client',
 'HTTPError',
 'MeshNeuron',
 'NeuronCriteria',
 'NeuronList',
 'NeuronModel',
 'Ra_HIGH',
 'Ra_LOW',
 'Ra_MED',
 'Rm_HIGH',
 'Rm_LOW',
 'Rm_MED',
 'SegmentCriteria',
 'SynapseCriteria',
 'ThreadPoolExecutor',
 'TimingResult',
 'TreeNeuron',
 'Volume',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__fetch_mesh',
 '__fetch_skeleton',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'as_completed',
 'client',
 'config',
 'connection_table_to_matrix',
 'dedent',
 'default_client',
 'fetch_adjacencies',
 'fetch_all_rois',
 'fetch_common_connectivity',
 'fetch_custom',
 'fetch_custom_neurons',
 'fetch_downstream_orphan_tasks',
 'fetch_mesh_neuron',
 'fetch_meta',
 'fetch_neurons',
 'fetch_output_completeness',
 'fetch_primary_rois',
 'fetch_roi',
 'fetch_roi_hierarchy',
 'fetch_shortest_paths',
 'fetch_simple_connections',
 'fetch_skeleton',
 'fetch_skeletons',
 'fetch_synapse_connections',
 'fetch_synapses',
 'fetch_traced_adjacencies',
 'heal_skeleton',
 '

## Let's normalise the synapse numbers by PN input to our LHNs

In [48]:
pn_lh_matrix_norm = pn_lh_matrix.div(pn_lh_matrix.sum(axis = 0), axis = 1)
pn_lh_matrix_norm

type_post,5-HTPMPV03,AL-MBDL1,ALIN3,AOTU010,AOTU043,APL,AVLP003,AVLP004,AVLP010,AVLP012,...,mAL3A,mAL4,mAL5B,mAL6,mALB1,mALB2,mALB3,mALB4,mALD1,NaN
type_pre,,,,,,,,,,,,,,,,,,,,,
DA1_lPN,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.777778,...,0.0,0.0,0.0,0.0,0.084577,0.000000,0.0,0.0,0.0,NaN
DA1_vPN,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN
DA2_lPN,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.004975,0.000000,0.0,0.0,0.0,NaN
DA3_adPN,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN
DA4l_adPN,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VP5+Z_adPN,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN
VP5+_l2PN,0.0,0.019608,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.042857,0.0,0.0,0.0,NaN
V_ilPN,0.0,0.117647,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.004975,0.000000,0.0,0.0,0.0,NaN


### Next, we want to take the LHNs we've got here and get a connection matrix between them and DNs. The crucial point to remember here is that the number of LHN rows must match the number of columns in the above data frame. Maybe I can collapsed into types 'after' the matrix multiplication, but I'm pretty sure this would mess with the calculation. 

In [67]:
from neuprint import fetch_adjacencies, NeuronCriteria as NC

# Example: Fetch all downstream connections FROM a set of neurons
LH_neuron_df, LH_conn_df = fetch_adjacencies(list(pn_lh_matrix_norm.columns), None)

#Here I've specified type, but the output is still bodyids...has it taken all bids that match the types provided? What is the difference in length if I specify bids? 
#Massive, 12000 against 29000. I guess what happens is if I use the bids we're only talking about those cells, whereas if we use types we're talking about all bids tht fall under those types.
#


KeyboardInterrupt: 

In [ ]:
LH_conn_df.sort_values('weight', ascending=False)

# Trying again using Philipp's functions and Alex's list of LHONs from R...this is probably the better approach for now